<a href="https://colab.research.google.com/github/gvsmaneesha/t5_training/blob/main/Training_Instance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd ~/../content

/content


In [2]:
!rm -rf t5_training
!git clone https://github.com/gvsmaneesha/t5_training.git

Cloning into 't5_training'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 66 (delta 29), reused 43 (delta 17), pack-reused 0
Unpacking objects: 100% (66/66), done.


In [3]:
cd t5_training

/content/t5_training


In [4]:
ls -l

total 112
drwxr-xr-x 2 root root  4096 Feb 22 07:58 config/
drwxr-xr-x 2 root root  4096 Feb 22 07:59 datasets/
drwxr-xr-x 2 root root  4096 Feb 22 07:59 learning_modules/
-rw-r--r-- 1 root root  2479 Feb 22 07:59 main.py
-rw-r--r-- 1 root root    47 Feb 22 07:58 README.md
-rw-r--r-- 1 root root   102 Feb 22 07:59 requirements.txt
-rw-r--r-- 1 root root 89368 Feb 22 07:58 Training_Instance.ipynb


In [5]:
!pip install -r requirements.txt

# New Section

In [6]:
print("imported all modules...")
import yaml
from pandas import read_csv
import pytorch_lightning  as pl
import argparse
import numpy as np
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics
from torch.utils.data import Dataset, DataLoader
from learning_modules.t5_tuning_modules import  T5Tokenizer,TweetDataset,LoggingCallback,T5FineTuner
print("successfully imported all modules")

with open('config/configuration.yaml',"r") as f:
        config = yaml.load(f,Loader=yaml.FullLoader)

print(config)
config["args_dict"]["learning_rate"] = float(config["args_dict"]["learning_rate"])
config["args_dict"]["adam_epsilon"] = float(config["args_dict"]["adam_epsilon"])

imported all modules...
importing t5 tuning modules
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
successfully imported all modules
{'args_dict': {'data_dir': 'datasets/', 'output_dir': 'outputs/', 'model_name_or_path': 't5-base', 'tokenizer_name_or_path': 't5-base', 'max_seq_length': 512, 'learning_rate': '3e-4', 'weight_decay': 0.0, 'adam_epsilon': '1e-8', 'warmup_steps': 0, 'train_batch_size': 8, 'eval_batch_size': 8, 'num_train_epochs': 1, 'gradient_accumulation_steps': 1, 'n_gpu': 0, 'early_stop_callback': False, 'fp_16': False, 'opt_level': 'O1', 'max_grad_norm': 1.0, 'seed': 42}, 'train_params': {'accumulate_grad_batches': 'args.gradient_accumulation_steps,', 'gpus': 'args.n_gpu,', 'max_epochs': 'args.num_train_epochs,', 'precision': '16 if args.fp_16 else 32,', 'amp_level': 'args.opt_level,', 'gradient_clip_val': 'args.max_grad_norm,'}}


In [7]:
print("training data import")
train_df = read_csv("datasets/tweet_dataset.csv")
train_df = train_df[["textID","text","selected_text","sentiment"]]
train, validate, test = np.split(train_df.sample(frac=1), [int(.6 * len(train_df)), int(.8 * len(train_df))])
train.to_csv("datasets/train.csv",index=False)
validate.to_csv("datasets/val.csv",index=False)
test.to_csv("datasets/test.csv",index=False)

training data import


In [8]:
#token initializer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

#load dataset
dataset = TweetDataset(tokenizer, data_dir='datasets/', type_path='val')
print("Length of dataset is :",len(dataset))

data = dataset[69]
print(tokenizer.decode(data['source_ids']))
print(tokenizer.decode(data['target_ids']))
args = argparse.Namespace(**config["args_dict"])

checkpoint_callback = pl.callbacks.ModelCheckpoint(
                dirpath=args.output_dir, monitor="val_loss", mode="min", save_top_k=5
        )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['textID', 'text', 'selected_text', 'sentiment']


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Length of dataset is : 8000
text: Fritz Roderick Von Hughes is sad to be in the kennel all day http://apps.facebook.com/dogbook/profile/view/6882699 sentiment: worry</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <p

In [9]:
train_params = config["train_params"]
train_params["callbacks"] = [LoggingCallback()]
train_params["checkpoint_callback"]=checkpoint_callback
print(train_params)

{'accumulate_grad_batches': 'args.gradient_accumulation_steps,', 'gpus': 'args.n_gpu,', 'max_epochs': 'args.num_train_epochs,', 'precision': '16 if args.fp_16 else 32,', 'amp_level': 'args.opt_level,', 'gradient_clip_val': 'args.max_grad_norm,', 'callbacks': [<learning_modules.t5_tuning_modules.LoggingCallback object at 0x7fe86c9aa8d0>], 'checkpoint_callback': <pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7fe86c758f90>}


In [10]:
train_params = dict(
                accumulate_grad_batches=args.gradient_accumulation_steps,
                gpus=args.n_gpu,
                max_epochs=args.num_train_epochs,
                precision=16 if args.fp_16 else 32,
                amp_level=args.opt_level,
                gradient_clip_val=args.max_grad_norm,
                checkpoint_callback=checkpoint_callback,
                callbacks=[LoggingCallback()]
        )
print(config["args_dict"])

# Model Intializer
model = T5FineTuner(args)

#trainer
trainer = pl.Trainer(**train_params)

{'data_dir': 'datasets/', 'output_dir': 'outputs/', 'model_name_or_path': 't5-base', 'tokenizer_name_or_path': 't5-base', 'max_seq_length': 512, 'learning_rate': 0.0003, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'train_batch_size': 8, 'eval_batch_size': 8, 'num_train_epochs': 1, 'gradient_accumulation_steps': 1, 'n_gpu': 0, 'early_stop_callback': False, 'fp_16': False, 'opt_level': 'O1', 'max_grad_norm': 1.0, 'seed': 42}


GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [ ]:
train_df

In [11]:
%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


In [12]:
trainer.fit(model)


  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


['textID', 'text', 'selected_text', 'sentiment']


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: T

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metric to the logger and/or progress bar (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metri

['textID', 'text', 'selected_text', 'sentiment']


Training: 0it [00:00, ?it/s]

RuntimeError: ignored

In [ ]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

In [ ]:
dataset =  TweetDataset(tokenizer, data_dir='datasets/', type_path='val')
loader = DataLoader(dataset, batch_size=32, num_workers=4)

In [ ]:
model.model.eval()
outputs = []
targets = []
for batch in tqdm(loader):
  outs = model.model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=200)

  dec = [tokenizer.decode(ids) for ids in outs]
  target = [tokenizer.decode(ids) for ids in batch["target_ids"]]
  
  outputs.extend(dec)
  targets.extend(target)